# Import libraries

In [1]:
from helpers import SEABORN_COLORS, create_plot
import os
import sys
import time
import datetime

from termcolor import colored, cprint

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sys
from os import listdir

import numpy as np
import pandas as pd  # Import Pandas and Numpy to create databases
import tensorflow as tf  # Import tensorflow to do tensor manipulations
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import csv
import statsmodels.api as sm
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# PATHS

In [2]:
PATH_data = f"/mnt/d/UKB_toydata/"
PATH_GENO = f"{PATH_data}genotype"
PATH_usefull = f"{PATH_data}usefull_values"
PATH_output = f"{PATH_data}usefull_values_2"
os.system(f"mkdir {PATH_output}")

mkdir: cannot create directory ‘/mnt/d/UKB_toydata/usefull_values_2’: File exists


256

# Start loop

In [3]:
diseases = listdir(PATH_usefull)

In [5]:
PATH_input = PATH_GENO+ "/02_per_chrom/"
name = "total_pop"
chrom = 22
plink_snps_pool = 20
os.system(f"plink2 --bfile {PATH_input}{name}_chr_{chrom} --maf 0 --max-maf 0.01 --mac 50 --geno 0.05 --recode A --out ./plink")
id_samples = pd.read_csv('./plink.raw', sep="\t")[['FID']]
os.system("rm plink.raw")


# NO CORRECTION
os.system(f"mkdir {PATH_output}/no_corr")
temp_path = f"{PATH_output}/no_corr/"



chroms = [21,22]
for disease in diseases:
    PATH_pheno = f"/mnt/d/UKB_toydata/"
    PATH_pheno = PATH_pheno + f"old/{disease}_eur_sample0.5.fam"
    columns = ['IID','FID','na1','na2','na3','phenotype']
    pheno_with_id = pd.read_csv(PATH_pheno, sep="\t", header=None, names=columns)
    #pheno_with_id = pd.read_csv(PATH_pheno, sep="\t", header=None, names=columns)[['FID','phenotype']]
    pheno = id_samples.merge(pheno_with_id, how='inner', on='FID')
    
    PRS = np.array([0] * pheno.shape[0])
    snps = []
    coefs = []
    std_errs = []
    ts = []
    Ps = []
    twfives = []
    ninfives = []
    os.system(f"mkdir {temp_path}/{disease}")
    dims = 100
    for chrom in chroms:
        path_genos = PATH_GENO
        path_genos = path_genos + "/04_LD_blocks/" + f"{dims}_PCs/chrom_{chrom}/"
        for file in listdir(path_genos):
            path_geno = path_genos + file
            genos = pd.read_pickle(path_geno)
            genos['FID'] = id_samples['FID']
            genos = genos.merge(pheno_with_id[['FID']], how='inner', on='FID')
            genos = genos.drop(columns='FID')
            for snp_id in list(genos.columns):
                #print('-------------------')
                #print(snp_id)
                #print(genos[snp_id])
                X2 = sm.add_constant(genos[[snp_id]])
                est = sm.OLS(pheno[['phenotype']], X2)
                est2 = est.fit()
                snps.append(est2.pvalues.index[1])
                coefs.append(est2.params[1])
                std_errs.append(est2.bse[1])
                ts.append(est2.tvalues[1])

                P = est2.pvalues[1]
                logp = - np.log(P)
                if logp > 0:
                    PRS = PRS + np.array(est2.tvalues[1]*genos[snp_id])
                else:
                    pass
                Ps.append(P)
                twfives.append(est2.conf_int()[0][1])
                ninfives.append(est2.conf_int()[1][1])
    sumstat = pd.DataFrame(data={
                'snp': snps,
                'coef': coefs,
                'std_err': std_errs,
                'Z': ts,
                'P': Ps,
                '0.025': twfives,
                '0.975': ninfives
            })
    sumstat['#CHROM'] = sumstat['snp'].str[:2].astype(int)
    sumstat['POS'] = [i.split(':', 1)[0] for i in list(sumstat['snp'].str[3:-5])]
    sumstat['POS'] = sumstat['POS'].astype(int)
    sumstat['-logp'] = - np.log(sumstat['P'])
    
    sumstat['#CHROM'] = [f.split(':')[0] for f in list(sumstat['snp'])]
    sumstat['POS'] = [f.split(':')[1] for f in list(sumstat['snp'])]
    sumstat['POS'] = sumstat['POS'].astype(int)
    running_pos = 0
    cumulative_pos = []
    for chrom, group_df in sumstat.groupby('#CHROM'):
        cumulative_pos.append(group_df['POS'] + running_pos)
        running_pos += group_df['POS'].max()

    sumstat['cumulative_pos'] = pd.concat(cumulative_pos)
    sumstat['SNP number'] = sumstat.index
    pheno['PRS'] = PRS
    sumstat.to_pickle(f"{temp_path}/{disease}/sumstat.pkl")
    pheno.to_pickle(f"{temp_path}/{disease}/pheno_PRS.pkl")
    

PLINK v2.00a5LM 64-bit Intel (21 Jun 2023)     www.cog-genomics.org/plink/2.0/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ./plink.log.
Options in effect:
  --bfile /mnt/d/UKB_toydata/genotype/02_per_chrom/total_pop_chr_22
  --export A
  --geno 0.05
  --mac 50
  --maf 0
  --max-maf 0.01
  --out ./plink

Start time: Thu Sep 14 20:23:59 2023
7857 MiB RAM detected, ~6213 available; reserving 3928 MiB for main workspace.
Using up to 12 threads (change this with --threads).
69455 samples (0 females, 0 males, 69455 ambiguous; 69455 founders) loaded from
/mnt/d/UKB_toydata/genotype/02_per_chrom/total_pop_chr_22.fam.
16775 variants loaded from
/mnt/d/UKB_toydata/genotype/02_per_chrom/total_pop_chr_22.bim.
Note: No phenotype data present.
Calculating allele frequencies... done.
--geno: 2 variants removed due to missing genotype data.
16649 variants removed due to allele frequency threshold(s)
(--maf/--max-maf/--mac/--max-mac).
124 variants remaining

mkdir: cannot create directory ‘/mnt/d/UKB_toydata/usefull_values_2/no_corr’: File exists
mkdir: cannot create directory ‘/mnt/d/UKB_toydata/usefull_values_2/no_corr//bc’: File exists


In [ ]:
# Abyss CORRECTION
os.system(f"mkdir {PATH_output}/no_corr")
temp_path = f"{PATH_output}/no_corr/"

PRS = np.array([0] * pheno.shape[0])
snps = []
coefs = []
std_errs = []
ts = []
Ps = []
twfives = []
ninfives = []
dims = [1, 2, 7, 15, 25, 100]
for dim in dims:
    chroms = [21,22]
    for disease in diseases[-2:-1]:
        PATH_pheno = f"/mnt/d/UKB_toydata/"
        PATH_pheno = PATH_pheno + f"old/{disease}_eur_sample0.5.fam"
        columns = ['IID','FID','na1','na2','na3','phenotype']
        pheno_with_id = pd.read_csv(PATH_pheno, sep="\t", header=None, names=columns)
        #pheno_with_id = pd.read_csv(PATH_pheno, sep="\t", header=None, names=columns)[['FID','phenotype']]
        pheno = id_samples.merge(pheno_with_id, how='inner', on='FID')
        os.system(f"mkdir {temp_path}/{disease}")
        dims = 100
        for chrom in chroms:
            path_genos = PATH_GENO
            path_genos = path_genos + "/04_LD_blocks/" + f"{dim}_PCs/chrom_{chrom}/"
            for file in listdir(path_genos)[0:2]:
                path_geno = path_genos + file
                genos = pd.read_pickle(path_geno)
                genos['FID'] = id_samples['FID']
                genos = genos.merge(pheno_with_id[['FID']], how='inner', on='FID')
                genos = genos.drop(columns='FID')
                for snp_id in list(genos.columns)[0:100]:
                    #print('-------------------')
                    #print(snp_id)
                    #print(genos[snp_id])
                    X2 = sm.add_constant(genos[[snp_id]])
                    est = sm.OLS(pheno[['phenotype']], X2)
                    est2 = est.fit()
                    snps.append(est2.pvalues.index[1])
                    coefs.append(est2.params[1])
                    std_errs.append(est2.bse[1])
                    ts.append(est2.tvalues[1])

                    P = est2.pvalues[1]
                    logp = - np.log(P)
                    if logp > 0:
                        PRS = PRS + np.array(est2.tvalues[1]*genos[snp_id])
                    else:
                        pass
                    Ps.append(P)
                    twfives.append(est2.conf_int()[0][1])
                    ninfives.append(est2.conf_int()[1][1])
        sumstat = pd.DataFrame(data={
                    'snp': snps,
                    'coef': coefs,
                    'std_err': std_errs,
                    'Z': ts,
                    'P': Ps,
                    '0.025': twfives,
                    '0.975': ninfives
                })
        sumstat['#CHROM'] = sumstat['snp'].str[:2].astype(int)
        sumstat['POS'] = [i.split(':', 1)[0] for i in list(sumstat['snp'].str[3:-5])]
        sumstat['POS'] = sumstat['POS'].astype(int)
        sumstat['-logp'] = - np.log(sumstat['P'])

        sumstat['#CHROM'] = [f.split(':')[0] for f in list(sumstat['snp'])]
        sumstat['POS'] = [f.split(':')[1] for f in list(sumstat['snp'])]
        sumstat['POS'] = sumstat['POS'].astype(int)
        running_pos = 0
        cumulative_pos = []
        for chrom, group_df in sumstat.groupby('#CHROM'):
            cumulative_pos.append(group_df['POS'] + running_pos)
            running_pos += group_df['POS'].max()

        sumstat['cumulative_pos'] = pd.concat(cumulative_pos)
        sumstat['SNP number'] = sumstat.index
        pheno['PRS'] = PRS
        sumstat.to_pickle(f"{temp_path}/{disease}/sumstat.pkl")
        pheno.to_pickle(f"{temp_path}/{disease}/pheno_PRS.pkl")